# Animation

In [1]:
import os
from skyfield.api import load
from tqdm import tqdm
from subprocess import Popen, PIPE
import pandas as pd
from plotting import plot_ground_path
from ground_path import ground_path
import matplotlib.pyplot as plt
import cartopy
from datetime import datetime

### Load data

In [2]:
resource_url = 'http://celestrak.com/satcat/tle.php?CATNR=40299'

fname = 'tle_hodoyoshi_1.txt'
satellites = load.tle(resource_url, filename=fname, reload=False)

satellite = satellites['HODOYOSHI 1']
print(satellite)

EarthSatellite 'HODOYOSHI 1' number=40299 epoch=2018-10-24T12:03:08Z


In [3]:
# generate ground path
date_from = datetime(2018,8,1,0,0,0,0)
date_to = datetime(2018,8,2,0,0,0,0)
freq = 60 # generate data point every 60 seconds
df = ground_path(satellite, date_from, date_to, freq)
df.head()

,X,Y,Z,lat,lon,elevation,ascending,orbit
datetime,,,,,,,,
2018-08-01 00:00:00+00:00,554.203355,-3178.014692,6054.771852,62.118186,-29.613950,499.094947,True,0
2018-08-01 00:01:00+00:00,364.781322,-2809.061221,6247.800731,65.759775,-32.392397,499.569012,True,0
2018-08-01 00:02:00+00:00,173.745475,-2427.680857,6413.112555,69.344170,-35.993558,500.002625,True,0
2018-08-01 00:03:00+00:00,-18.059004,-2035.561712,6549.976432,72.836863,-40.906509,500.383247,True,0
2018-08-01 00:04:00+00:00,-209.783681,-1634.439031,6657.787568,76.169689,-48.043976,500.700213,True,0


### Create frames

In [4]:
frames_directory = './frames'
if not os.path.exists(frames_directory):
    os.makedirs(frames_directory)

In [5]:
for i in tqdm(range(1, len(df), 5)):
    df_sub = df.iloc[:i]
    dt = df_sub.index[-1]
    
    title = "HODOYOSHI-1\n{:} ~ {:}".format(df_sub.index.min(), df_sub.index.max())

    fig = plt.figure(figsize=(20,10))
    # map projection
    projection0 = cartopy.crs.PlateCarree()
    # object transformation
    transform = cartopy.crs.Geodetic()
    ax0 = fig.add_subplot(1,1,1, projection=projection0)
    plot_ground_path(ax0, df_sub, transform=transform)
    ax0.set_title(title)
    ax0.legend(loc='upper left')
    plt.savefig(frames_directory + '/hodoyoshi_1_{:%Y%m%d%H%M%S}.jpg'.format(dt))
    plt.cla()   # Clear axis
    plt.clf()   # Clear figure
    plt.close() # Close a figure window

100%|██████████| 288/288 [01:41<00:00,  2.83it/s]


In [6]:
df.to_csv('hodoyoshi_1_2018.csv')

### Create video

In [7]:
def make_video(in_dir, out_dir):
    p = Popen(['./make_video.sh "{:}" "{:}"'.format(in_dir, out_dir)], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    output, err = p.communicate()
    print(p.returncode)
    #print(output.decode('utf-8'), err.decode('utf-8'))
    del p
    return None

In [8]:
videos_directory = './videos'
if not os.path.exists(videos_directory):
    os.makedirs(videos_directory)

In [9]:
make_video(frames_directory + '/hodoyoshi_1_*.jpg', videos_directory + '/hodoyoshi_1_2018.mp4')

0
